## Implementación el método de la máxima verosimulitud para la regresión logística
### Definir la función de entorno L(b) 

In [1]:
from IPython.display import display, Math, Latex
import numpy as np
import pandas as pd
display(Math(r"L(\beta)=\sum_{i=1}^n P_i^{y_i}(1-Pi)^{y_i}"))

<IPython.core.display.Math object>

In [2]:
def linkedlihood(y: list, pi):
    import numpy as np 
    total_sum=1
    sum_in = range(1, len(i+1))
    for i in range(len(y)):
        sum_in[i]=np.where(y[i]==1, pi[i], 1-pi[i])
        total_sum=total_sum*sum_in[i]
    return total_sum
def likelihood(y, pi):
    import numpy as np
    total_sum = 1
    sum_in = list(range(1, len(y)+1))
    for i in range(len(y)):
        sum_in[i] = np.where(y[i]==1, pi[i], 1-pi[i])
        total_sum = total_sum * sum_in[i]
    return total_sum

## Calcular las probabilidades para cada observación

In [3]:
print("Al hacer P(xi) significa hacer esa fórmula para cada uno de los valores")
display(Math(r"P_i=P(x_i)=\frac{1}{1+e^{-\sum_{j=0}^k\beta_j\cdot x_j}}"))
print("K es número de columnas")

Al hacer P(xi) significa hacer esa fórmula para cada uno de los valores


<IPython.core.display.Math object>

K es número de columnas


In [4]:
import time

In [5]:
def logitprobs3(X, beta):
    import numpy as np
    n_rows=np.shape(X)[0]#obtener el número de filas
    n_cols=np.shape(X)[1]#obtener el número de columnas
    pi=range(n_rows+1)#crea un array desde la posición 1 hasta el número de filas
    expon=range(1, n_rows)
    for i in range(n_rows):# recorremos el número de filas 
        expon[i]=0#el valor del exponene es j y empieza en 0, recorremos columnas
        ## obtener la súmatoria del exponente
        for j in range(n_cols):
            ex=X[i][j]*beta[j]##accedemos al dato columna y fila y lo multiplicamos por un array de beta
            expon[i]=ex+expon[i]## calculo el exponente y los sumo a los anteriores para hacer una sumatoria
        with np.errstate(divide="ignore", invalid="ignore"):
            ##realizamos la formula
            pi[i]=1/(1+np.exp(-expon[i])) # expon[i]por que ahí fuimos sumando los valores 
    return pi
def logitprobs(X,beta):
    print("beta: ",beta)
    import numpy as np
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi=list(range(1,n_rows+1))
    expon=list(range(1,n_rows+1))
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex=X[i][j] * beta[j]
            print("ex", ex, "fin ex")
            expon[i] = ex + expon[i]
        with np.errstate(divide="ignore", invalid="ignore"):
            pi[i]=1/(1+np.exp(-expon[i]))
    return pi

## Calcular la matriz diagonal W

In [6]:
display(Math(r"W=diag(P_i  \cdot (1-P_i))_{1=1}^n"))
print("n es el número de datos del dataset original")

<IPython.core.display.Math object>

n es el número de datos del dataset original


In [7]:
def findW2(pi):
    import numpy as np 
    n=len(pi)##el tamaño de los datos
    W=np.zeros(n*n).reshape(n,n)
    for i in range(n):
        ## creamos una diagonal de valores
        W[i,i]=pi[i]*(1-pi[i])##hacemos la fórmula n i=1
    return W
def findW(pi):
    import numpy as np
    n = len(pi)
    W = np.zeros(n*n).reshape(n,n)
    for i in range(n):
        print(i)
        W[i,i]=pi[i]*(1-pi[i])
        W[i,i].astype(float)
    return W

## Obtener la solución de funcion logística

In [8]:
display(Math(r"x_{n+1}=x_n-\frac{f(x_n)}{f'(x_n)}"))
display(Math(r"f(X)=X(Y-P)"))
display(Math(r"f'(X)=XWX^T"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [9]:
def logistics(X, Y, limit):
    import numpy as np
    from numpy import linalg
    nrow = np.shape(X)[0]
    bias = np.ones(nrow).reshape(nrow,1)
    X_new = np.append(X, bias, axis = 1)
    print("Inicio", X_new, "fin")
    ncol = np.shape(X_new)[1]
    beta = np.zeros(ncol).reshape(ncol,1)
    root_dif = np.array(range(1,ncol+1)).reshape(ncol,1)
    iter_i = 10000
    while(iter_i>limit):
        print("Iter:i"+str(iter_i) + ", limit:" + str(limit))
        print("beta in", beta, "beta fin")
        pi = logitprobs(X_new, beta)
        print("Pi:"+str(pi))
        W = findW(pi)
        print("W:"+str(W))
        num = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose())
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new))
        root_dif = np.array(linalg.inv(den)*num)
        beta = beta + root_dif
        print("Beta: "+str(beta))
        iter_i = np.sum(root_dif*root_dif)
        ll = likelihood(Y, pi)
    return beta

In [10]:
X=np.arange(10).reshape(10,1)
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [11]:
Y= [0,0,0,0,1,0,1,0,1,1]
bias = np.ones(10).reshape(10,1)
X_new = np.append(X,bias,axis=1)

In [12]:
a=logistics(X, Y, 0.00001)

Inicio [[0. 1.]
 [1. 1.]
 [2. 1.]
 [3. 1.]
 [4. 1.]
 [5. 1.]
 [6. 1.]
 [7. 1.]
 [8. 1.]
 [9. 1.]] fin
Iter:i10000, limit:1e-05
beta in [[0.]
 [0.]] beta fin
beta:  [[0.]
 [0.]]
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
ex [0.] fin ex
Pi:[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
W:[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.

In [13]:
ll = likelihood(Y, logitprobs(X,a))

beta:  [[ 0.66220827]
 [-3.69557172]]
ex [0.] fin ex
ex [0.66220827] fin ex
ex [1.32441653] fin ex
ex [1.9866248] fin ex
ex [2.64883307] fin ex
ex [3.31104134] fin ex
ex [3.9732496] fin ex
ex [4.63545787] fin ex
ex [5.29766614] fin ex
ex [5.95987441] fin ex


In [14]:
preddicion=-3.69557172 +X* 0.66220827

In [15]:
preddicion

array([[-3.69557172],
       [-3.03336345],
       [-2.37115518],
       [-1.70894691],
       [-1.04673864],
       [-0.38453037],
       [ 0.2776779 ],
       [ 0.93988617],
       [ 1.60209444],
       [ 2.26430271]])

In [16]:
import statsmodels.api as sm
import pandas as pd
from pandas import Timestamp

In [17]:
Y = (Y - np.min(Y))/np.ptp(Y)
logit_model = sm.Logit(Y,X_new)

In [18]:
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [19]:
print(result.summary2())

                        Results: Logit
Model:              Logit            Pseudo R-squared: 0.360   
Dependent Variable: y                AIC:              12.6202 
Date:               2022-06-25 11:13 BIC:              13.2254 
No. Observations:   10               Log-Likelihood:   -4.3101 
Df Model:           1                LL-Null:          -6.7301 
Df Residuals:       8                LLR p-value:      0.027807
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     6.0000                                     
-----------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
-----------------------------------------------------------------
x1        0.6622     0.4001    1.6551   0.0979   -0.1220   1.4464
const    -3.6956     2.2889   -1.6145   0.1064   -8.1818   0.7906



In [20]:
preddicciones=result.predict(X_new)

In [21]:
def divisor(entrada):
    res=[]
    for n in entrada:
        if n>0.5:
            res.append(1)
        else:
           res.append(0)
    return res

In [22]:
res=divisor(list(preddicciones))
res, Y

([0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 array([0., 0., 0., 0., 1., 0., 1., 0., 1., 1.]))

In [23]:
ll = likelihood(Y, logitprobs(X,a))

beta:  [[ 0.66220827]
 [-3.69557172]]
ex [0.] fin ex
ex [0.66220827] fin ex
ex [1.32441653] fin ex
ex [1.9866248] fin ex
ex [2.64883307] fin ex
ex [3.31104134] fin ex
ex [3.9732496] fin ex
ex [4.63545787] fin ex
ex [5.29766614] fin ex
ex [5.95987441] fin ex
